# LLMを使用したテキストの要約

LLMは言語を理解するため、翻訳や要約などのタスクに適しています。  
例えばこのNotebookでは、LLMを使用して、保険の請求に関する文章を要約します。

### 必要なライブラリとインポート

Labの指示に従って適切なワークベンチイメージを選択して起動した場合、必要なすべてのライブラリがすでにインストールされているはずです。もしインストールされていない場合は、次のセルの最初の行のコメントを外して正しいパッケージをすべてインストールしてください。その後、必要なライブラリをインポートします。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 正しいワークベンチイメージを選択していない場合のみ、コメントを外してください

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

### Langchainパイプライン

Langchainを使用して、要約のパイプラインを定義します。

In [ ]:
# LLM推論APIのURL
inference_server_url = "_INFERENCE_URL_LLM_"

# LLMの定義
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base= f"{inference_server_url}/v1",
    model_name="elyza",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

以下は、要約タスクに対してフォーマットされた**テンプレート**です。

In [ ]:
system_template_string = """
あなたは、親切で、礼儀正しく、正直なアシスタントです。
常に気配りと尊重をもって接し、真摯にサポートします。できる限り有用な返答を提供しますが、安全を確保します。
有害で、倫理に反する、偏見のある、または否定的な内容は避けます。返答が公正でポジティブなものであることを確認します。
"""

user_template_string = """
提供された文章を要約して下さい。要約は5項目以内の箇条書きにしてください。

### 文章:
{input}

### 要約:
"""

system_template = SystemMessagePromptTemplate.from_template(system_template_string)
user_template = HumanMessagePromptTemplate.from_template(user_template_string)

PROMPT = ChatPromptTemplate.from_messages([system_template, user_template])

モデルにクエリを投げるために使用する**会話**オブジェクトを作成します。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

モデルにクエリする準備が整いました。

`claims`フォルダーには、受信される可能性のある請求文章の例がJSONファイルで保存されています。これらのファイルを読み込んで表示し、その後にLLMが作成した要約を表示します。

In [ ]:
# 請求文章の読み取り
claims_path = 'claims'
onlyfiles = [f for f in listdir(claims_path) if isfile(join(claims_path, f))]

claims = {}

for filename in onlyfiles:
    with open(os.path.join(claims_path, filename), 'r') as file:
        data = json.load(file)
    claims[filename] = data

In [ ]:
for filename in onlyfiles:
    print(f"***************************")
    print(f"* 請求: {filename}")
    print(f"***************************")
    print("元の文章:")
    print("-----------------")
    print(f"件名: {claims[filename]['subject']}\n内容:\n{claims[filename]['content']}\n\n")
    print('要約:')
    print("--------")
    summary_input = f"件名: {claims[filename]['subject']}\n内容:\n{claims[filename]['content']}"
    conversation.predict(input=summary_input);
    print("\n\n                          ----====----\n")